In [96]:
import pandas as pd
import numpy as np
import sys, os

# Pull in Observed Solar Radiation Data

Columns descriptions found here: ftp://aftp.cmdl.noaa.gov/data/radiation/solrad/README_SOLRAD.txt

In [97]:
solar_rad_2016_2017_df = pd.DataFrame()
for year in [2016, 2017]:
    for i in range(1, 366):
        curr_df = pd.read_csv('data/aftp.cmdl.noaa.gov/data/radiation/solrad/hnx/' + str(year) + '/hnx17' + str(i).zfill(3) + '.dat', sep='\s+', header=None, skiprows=2)
        curr_df = curr_df.rename({0: "year", 1: "jday", 2: "month", 3: "day", 4:"hour", 5:"min",
                               6:"dt", 7:"zen", 8:"dw_psp", 9:"direct", 10:"diffuse", 11:"uvb",
                               12: "uvb_temp", 13:"qc_dwpsp", 14:"qc_direct", 15:"qc_diffuse",
                               16:"qc_uvb", 17:"qc_uvb_temp", 18:"std_dw_psp", 19:"std_direct",
                               20:"std_diffuse", 21:"std_uvb"}, axis='columns')
        curr_df = curr_df.groupby(by=["hour", "year", "jday", "month", "day"]).mean()
        solar_rad_2016_2017_df = pd.concat([solar_rad_2016_2017_df, curr_df])

FileNotFoundError: File b'data/aftp.cmdl.noaa.gov/data/radiation/solrad/hnx/2016/hnx17001.dat' does not exist

In [95]:
print(solar_rad_2016_2017_df.shape)
solar_rad_2016_2017_df.head()

(0, 0)


""


In [93]:
#solar_rad_2017_df["dw_psp"].unique()